## Get Packages

In [1]:
!nvidia-smi

Wed Aug 31 09:35:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np

import matplotlib.pyplot as plt

import tensorflow as tf

from skimage.io import imread
from skimage import img_as_float
from skimage.color import rgb2gray
from skimage.util import random_noise

from scipy import signal

from tensorflow import keras
from tensorflow.keras import layers

import xarray as xr

import io
import imageio
from IPython.display import Image, display
from ipywidgets import widgets, Layout, HBox

## Get the Data

In [4]:
z500_dataset_path = "/content/drive/MyDrive/Machine Learning Project Files/Data/geopotential_500/*.nc"

temp_2m_dataset_path = "/content/drive/MyDrive/Machine Learning Project Files/Data/temperature_2m/*.nc"

solar_dataset_path = "/content/drive/MyDrive/Machine Learning Project Files/Data/toa_incident_solar_radiation/*.nc"

## Create Model

In [5]:
class Sampling(tf.keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


In [11]:
def create_encoder(latent_dim, batch_size=32, input_shape=(32, 64, 1),
                   number_of_filters=(32, 64, 128), k_shape=(3, 3, 3), 
                   stride=(2, 2, 2), LSTM_units = 64*64 ):
    encoder_inputs = tf.keras.Input(shape=input_shape, batch_size=batch_size)
    x = encoder_inputs
    for i in range(len(number_of_filters)):
        x = tf.keras.layers.Conv2D(filters=number_of_filters[i],
                                   kernel_size=k_shape[i],
                                   activation='relu',
                                   strides=(stride[i], stride[i]),
                                   padding='same')(x)
    x = tf.keras.layers.Flatten()(x)

    # x = tf.keras.layers.Reshape((64, 64))(x)

    # x = tf.keras.layers.Dense(latent_dim + latent_dim)(x)

    # x = tf.keras.layers.LSTM(LSTM_units, stateful=True)(x) #LSTM LAYER

    x = tf.keras.layers.Dense(latent_dim + latent_dim)(x)

    z_mean = tf.keras.layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = tf.keras.layers.Dense(latent_dim, name="z_log_var")(x)
        
    z = Sampling()([z_mean, z_log_var])
    encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

    return encoder


In [12]:
def create_decoder(latent_dim, batch_size=32, input_shape=(32, 64, 1), number_of_filters=(32, 64, 128), k_shape=(3, 3, 3), stride=(2, 2, 2) ):
    latent_inputs = keras.Input(shape=(latent_dim,))
    x = tf.keras.layers.Dense( (input_shape[0]//2**(len(number_of_filters))) * (input_shape[1]//2**(len(number_of_filters))) * number_of_filters[-1], activation="relu")(latent_inputs)
    x = tf.keras.layers.Reshape( ((input_shape[0]//2**(len(number_of_filters))), (input_shape[1]//2**(len(number_of_filters))), number_of_filters[-1]))(x)

    for i in range(len(number_of_filters)):
        j = len(number_of_filters) - 1 - i
        x = tf.keras.layers.Conv2DTranspose(filters=number_of_filters[j],
                                            kernel_size=k_shape[j],
                                            activation='relu',
                                            strides=(stride[j], stride[j]),
                                            padding='same')(x)

    decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
    decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
    return decoder


In [13]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        data, target = data
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(target, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [14]:
latent_dim = 2
batch_size= 32
input_shape=(32, 64, 1)
number_of_filters=(32, 64)
k_shape=(3, 3)
stride=(2, 2) 

EPOCHS = 500

In [15]:
encoder = create_encoder(latent_dim,
                         batch_size=batch_size,
                         input_shape=input_shape,
                         number_of_filters=number_of_filters,
                         k_shape=k_shape,
                         stride=stride,
                         LSTM_units= 4*latent_dim)
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(32, 32, 64, 1)]    0           []                               
                                                                                                  
 conv2d_2 (Conv2D)              (32, 16, 32, 32)     320         ['input_2[0][0]']                
                                                                                                  
 conv2d_3 (Conv2D)              (32, 8, 16, 64)      18496       ['conv2d_2[0][0]']               
                                                                                                  
 flatten_1 (Flatten)            (32, 8192)           0           ['conv2d_3[0][0]']               
                                                                                            

In [17]:
decoder = create_decoder(latent_dim,
                         batch_size=batch_size,
                         input_shape=input_shape,
                         number_of_filters=number_of_filters,
                         k_shape=k_shape,
                         stride=stride )
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_2 (Dense)             (None, 8192)              24576     
                                                                 
 reshape_2 (Reshape)         (None, 8, 16, 64)         0         
                                                                 
 conv2d_transpose_3 (Conv2DT  (None, 16, 32, 64)       36928     
 ranspose)                                                       
                                                                 
 conv2d_transpose_4 (Conv2DT  (None, 32, 64, 32)       18464     
 ranspose)                                                       
                                                                 
 conv2d_transpose_5 (Conv2DT  (None, 32, 64, 1)        289 

## Train Model

In [18]:
z500 = xr.open_mfdataset(z500_dataset_path, combine='by_coords')

In [19]:
ds_train = z500.sel(time=slice('2015', '2016'))  
ds_test = z500.sel(time=slice('2017', '2018'))

In [20]:
dataset = np.array(ds_train.z)

In [21]:
num_samples = (dataset.shape[0]//batch_size)*batch_size + 1

In [22]:
def normalize_data(data):
    # expect shape (samples, height, width)
    v_min, v_max = data.min(), data.max()
    return ((data-v_min)/(v_max-v_min))

def expand_data(data):
    # expand dim
    return np.expand_dims(data, axis=-1)

def create_shifted_frames(data):
    #expect shape (samples, height, width, 1)
    x = data[0 : data.shape[0] - 1, :, :, :]
    y = data[1 : data.shape[0], :, :, :]
    return x, y


In [23]:
x, y = create_shifted_frames(expand_data(normalize_data(dataset[:num_samples,:,:])))

In [24]:
print(x.shape, y.shape)

(17536, 32, 64, 1) (17536, 32, 64, 1)


In [25]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam(1e-3))

In [26]:
batch_size = 32

In [ ]:
vae.fit(x=x, y=x, epochs=EPOCHS*2, batch_size=batch_size )

Epoch 1/1000
548/548 [==============================] - 16s 6ms/step - loss: 1106.5611 - reconstruction_loss: 1059.3059 - kl_loss: 2.4813
Epoch 2/1000
548/548 [==============================] - 3s 5ms/step - loss: 1042.5902 - reconstruction_loss: 1041.0189 - kl_loss: 1.7632
Epoch 3/1000
548/548 [==============================] - 3s 5ms/step - loss: 1043.1965 - reconstruction_loss: 1040.8657 - kl_loss: 1.7308
Epoch 4/1000
548/548 [==============================] - 3s 6ms/step - loss: 1042.3942 - reconstruction_loss: 1040.7760 - kl_loss: 1.7418
Epoch 5/1000
548/548 [==============================] - 3s 6ms/step - loss: 1042.6112 - reconstruction_loss: 1040.6914 - kl_loss: 1.7272
Epoch 6/1000
548/548 [==============================] - 3s 6ms/step - loss: 1042.4222 - reconstruction_loss: 1040.6049 - kl_loss: 1.7568
Epoch 7/1000
548/548 [==============================] - 3s 6ms/step - loss: 1042.7507 - reconstruction_loss: 1040.5327 - kl_loss: 1.7565
Epoch 8/1000
548/548 [==================

### Training Loss

In [ ]:
plt.plot(vae.history.history["loss"])